# 0 摘要

广义线性模型+非线性特征变换是现在大规模回归或者分类任务的常规配置。通过对特征进行交乘变换增加对特征交叉的memorization是非常有效的一种方法，但往往需要比较大的特征工程成本。

DNN可以通过对稀疏特征转换为低维稠密embedding，进而能够对泛化不同的特征组合；但是当user-item的交叉是稀疏且高秩的时候，DNN学到的embeddings会over-generalize并给用户推荐一些不相关的items。

本篇论文提出Wide & Deep Learning，同时训练wide线性模型和DNN，将memorization和generalization的优势结合起来。在Google Play的实验中取得了由于wide-only和deep-only的模型效果。

# 1 简介

一个推荐系统可以被看做是一个搜索排序系统，给定一个query（query是用户和上下文的集合），输出排序后的物品列表。

**推荐系统的两大挑战**：

- 1.Memorization: 通过历史数据，学习一些物品或者特征的高频共现信息以及他们的相关性，例如两个物品经常被一同购买；Memorization推荐的结果往往更热门，并且和用户已经有过行为的物品是直接相关的；
- 2.Generalization: 主要是对相关性的一个转移或推断，进而探索历史数据中没有出现过的新的特征组合信息；Generalization推荐的结果倾向于多样性；

**问题：**

在推荐系统中，逻辑回归是常用的算法，其会对特征进行OHE或二值化处理，例如```user_installed_app=netflix```为1代表用户安装了netflix，否则为0；

- Memorization可以有效地利用变量交叉来增加信息，例如```AND(user_installed_app=netflix, impression_app=pandora)```为1代表满足上述两个条件，否则为0；Memorization实际上反映了当两个特征共现时，其与目标的关系；但Memorization对于历史数据中没有出现过的共现特征没法学习权重；

- Generalization则是在更粗粒度的一个特征工程，例如```AND(user_installed_category=video, impression_category=music)```

**现有解决方案：**

如FM或FFM这种embedding-based models，可以通过学习低维稠密向量表示来泛化在样本中没有出现过的query-item feature pairs。

但是这种方法在query-item矩阵非常稀疏且高秩的情况下是难以学习到有效的向量表示的；例如用户-物品矩阵非常稀疏时，意味着用户的兴趣非常specific且小众的时候，绝大多数query-item pairs都没有出现过，但Dense embeedings会对所有的query-item pairs进行打分，预测一个非零的值，这个时候会导致over-generalize并且推荐很多低相关的结果；另一方面，线性模型的特征交乘可以使用更少的参数来memorize这些“exception rules”。

** 本文贡献：**

- 提出Wide&Deep学习框架，可以同时通过DNN训练embeddings和通过线性模型特征变化处理稀疏特征；
- Wide&Deep应用在google的手机app store上已经产品化；
- 开源了Wide&Deep的TensorFlow源代码。

![](pictures/wide-and-deep.png)

# 2 模型详解

## 2.1 推荐系统架构

推荐系统主要分为两个阶段：

- 召回。通过简单机器学习模型和人工规则，基于用户和上下文的query，召回物品集；
- 排序。通过模型算法对召回集进行排序。

![](pictures/recommender-system.png)

## 2.2 Wide部分

wide部分是一个广义线性模型（Generalized linear model）：

$$y = w^T x + b$$

输入包含两部分：
- 1.原始特征（Raw input features）
- 2.转换特征（Transformed features）

其中最重要的转换之一就是cross-product transformation，定义如下：

$$\phi_k(x)=\prod_{i=1}^d x_i^{c_{ki}}$$

其中$c_{ki}$是一个boolean类型，当第$i$个特征是第$k$个转换特征$\phi_k$的一部分时，$c_{ki}=1$，否则为0。例如对于binary features，令$\phi_k$为```AND(gender=female, language=en)```，那么只有当构成$\phi_k$的两个特征满足```(gender=female and language=en)```时，$\phi_k=1$。

这种转换可以捕捉不同binary特征的交叉信息，给线性模型增加非线性的学习能力。

> 对特征进行离散化+二值化可以给线性模型带来非线性学习能力：原因在于线性模型会对某一特征离散化后的每个区间段都学习到一个权重。距离来说，假设年龄这个特征和是否购买物品是“倒U型关系”，即年龄越大或年龄越小，用户购买的概率都越低。如果此时直接拿该特征进行学习，线性模型无法对拟合好这种关系；假设我们此时将年龄离散化为$(0,6], (6,12], (12,18], (18, 24], (24, 30], (30, 36], \cdots$等，可以构造出很多0-1变量，此时模型可以对每个区间学习一个权重，进而就可以来拟合原始特征中的“倒U型关系“。

## 2.3 Deep部分

Deep部分是一个feed-forward neural networks。对于类别变量，例如```language=en```，原始输入会作为OHE输入给模型，即是高维稀疏特征，Deep部分可以对每个稀疏特征的取值学习到一个低维稠密特征embeddings（类似Word2Vec）。一般学习到的embeddings的维度是几十维或几百维。

Embeddings的训练过程和Word2Vec类似，先初始化随机数，进而通过整个模型最小化损失函数的过程中来进行参数更新。

Embeddings后续会接入多层hidden layers：

$$a^{(l+1)}=f(W^{(l)}a^{(l)}+b^{(l)})$$

其中$f$为激活函数，一般使用ReLU。

## 2.4 联合训练Wide & Deep

wide与deep部分最后输出的向量进行concat后，再经过一层全连接层输出到一个结点得到log odds，进而使用sigmoid函数进行转化，得到最后的概率值。

Wide & Deep是joint training，而不是ensemble模型。ensemble模型的各个子模型都是独立学习，最后在inference阶段进行组合输出；而joint training是在训练阶段就同时训练所有模型参数。这两种方式在model size上的区别在于，emsemble模型由于各个子模型都是独立训练，就需要每个子模型model size比较大才能学习到更多的信息；而joint training，wide部分只需要通过少量的交乘特征来补充deep部分的不足就好，model size也相对需要较少。

在训练细节上：
- 用mini-batch SGD同时训练wide部分和deep部分；
- Wide部分的优化器采用带有L1正则的FTRL，主要是用来稀疏化权重，加速serving；
- Deep部分使用AdaGrad进行优化。

Wide和Deep的结合输出：

$$P(Y=1|x)=\sigma(w^T_{wide}[x, \phi(x)] + w^T_{deep}[a^{(l_f)}] + b)$$

# 3 工程实现流程

整个模型的实现与部署分为以下三个步骤：
- 1.Data Generation: 样本与特征构造
- 2.Model Training: 模型训练
- 3.Model Serving: 模型预测

## 3.1 Data Generation

在样本构造中，主要包含了一段时间内用户与app的曝光数据。每条样本对应一个曝光，目标Y是曝光app是否被用户安装。

**特征方面**：
- 1.ID类特征：对低频异常取值进行截断（即过滤掉出现次数小于阈值的app），对这些string进行映射为id，构造一个vocabulary；
- 2.连续实数特征：采用Quantile归一化，将样本归一到$[0,1]$区间内。具体来说，计算特征$x$的概率累积函数，切分为$n_q$个quantile，使用$\frac{i-1}{n_q-1}$进行归一化。

## 3.2 Model Training

模型整体架构如下：

- 1.Wide部分包括了对用户历史安装与曝光app的交乘特征，主要用来memorization。
- 2.Deep部分包括了连续实数特征和ID类特征的embedding，对这些特征进行concat获得1200维的dense features，再接了3层ReLU全连接层。

![](pictures/wide-and-deep-model-structure.png)

整个模型训练的样本超过5000亿条，并且采用了一种online-learning的架构，每新来一些数据会重新进行训练。

考虑到train from scratch的时间成本过高，作者提出了一种warm-starting system，用上一版模型的embeddings和权重来初始化新的模型，再基于新的数据进行fine-tune。

## 3.3 Model Serving

对于每个request，服务器会召回一部分app集合以及一些用户特征去对集合中的每个app预测分数。最后再按照预测结果ranking后呈现给用户。

考虑到对于每个request要在10ms中相应，在系统层面，通过并行运行较小batch，使用多线程并行性来优化性能，而不是在批量对整个app召回集进行打分。